# Part 4: Using GPU acceleration with PyTorch

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 138 kB 12.3 MB/s 


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [2]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[-5.5582e-01,  9.8506e-01, -1.2312e+00,  ...,  9.9659e-01,
         -3.3670e-01,  7.9248e-02],
        [-2.4446e-02, -1.3857e-01,  1.4147e+00,  ..., -1.5561e-01,
         -6.5432e-01, -1.0956e+00],
        [ 5.5027e-01,  2.5774e-01, -1.4111e+00,  ..., -6.0695e-01,
          6.8110e-01,  1.7106e-01],
        ...,
        [ 1.1238e-02, -6.4855e-02, -8.1168e-01,  ..., -4.1626e-02,
          2.5147e-02, -5.3411e-04],
        [ 1.8064e-04,  9.8615e-01,  9.0629e-03,  ...,  4.1099e-01,
          1.4054e+00, -7.9551e-01],
        [-1.4603e+00,  9.3961e-01,  8.4710e-01,  ...,  2.8001e+00,
          1.6934e+00,  2.4133e-01]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [3]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[ 2.7353e+00,  1.1655e+00, -1.9069e-02,  ..., -3.7231e-01,
         -4.8646e-01, -1.5439e-02],
        [ 1.5450e+00,  7.9112e-02,  8.4848e-01,  ...,  2.3113e-01,
         -9.8565e-02,  2.1596e-02],
        [-2.2474e-01, -1.9021e-02,  2.6352e-02,  ..., -3.1213e-02,
         -3.3373e-02,  1.5323e-01],
        ...,
        [ 7.4364e-01,  1.0850e+00,  1.9742e-02,  ..., -1.5520e+00,
         -6.7883e-01, -2.0290e-01],
        [-2.0967e-01, -1.2400e-03,  1.3443e-02,  ...,  3.2297e-01,
          5.5670e-01, -2.2818e-01],
        [ 1.8827e+00, -3.2880e-01, -4.7433e-01,  ...,  6.9318e-01,
         -6.2503e-01,  1.1836e+00]], device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-1.1131e+00,  1.0790e+00, -1.8110e+00,  ..., -1.4917e+00,
          4.1178e-02, -8.1272e-01],
        [-6.5117e-02, -2.5483e-01, -1.4996e+00,  ..., -8.2185e-03,
          1.4452e-03, -1.3973e+00],
        [-9.8326e-01, -3.7880e-01, -2.4808e-01,  ..., -9.4188e-01,
          1.7902e+00,  3.6644e-01],
        ...,
        [ 4.4730e-01, -2.4512e-01,  2.6187e-01,  ...,  6.3037e-01,
         -4.9489e-03, -1.7821e-01],
        [-8.7708e-01,  4.2919e-01,  7.8215e-01,  ..., -2.8335e-02,
          3.0368e-02, -1.0625e+00],
        [-3.9451e-01,  1.5431e-01,  1.0358e+00,  ..., -3.3488e-01,
         -6.3267e-01,  4.5839e-01]], device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

Memory Bandwidth is the main reason for GPU to perform better than CPU and parellel processing

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

Available memory of the compute hardware